# The SNOMED Graph
## Reading Format
The SNOMED Graph is designed as a Directed Acyclic Graph, with a single root node called the Root Concept, or `SNOMED CT Concept (SNOMED RT+CTV3) |138875005|`. Each concept in SNOMED is connected to one or more concepts through various types of relationships, such as `|Is a|`, `|Part of|`, `|Has AMP|` etc. as you'll see later on in this notebook. Due to its size, importance, and the computational difficulty of generating it, the `|Is a|` relationship subgraph (list of all `Is a` relationship pairs of the form `A |is a| B`) is denormalized to a format called the [`transitive closure`](https://en.wikipedia.org/wiki/Transitive_closure#In_graph_theory) of the `|Is a|` relationship subgraph. For example, if we have this relationship structure:

+ ```A |is a| B```
+ ```B |is a| C```
+ ```C |is a| D```

Then the transitive closure will look like this: 

*`[(A,B),(A,C),(A,D),(B,C),(B,D),(C,D)]`*.

This data structure easily answers the question: `can I reach D from A by asking whether I can reach B from A, then asking whether I can reach C from B?`. If there's a tuple `(A,D)`, then the answer is yes. If the tuple doesn't exist, then the answer is No.

This example graph, with three simple relationships, spawns a transitive closure with six connections. The SNOMED transitive closure has 15,592,960 connections between 903,389 concepts.

In order to do anything relevant with SNOMED, such as collecting a list of drugs in the DM+D hierarchy (see below), we need this structure because it enables quick and efficient discovery of the local 'friends' of any given concept.

To make it easier to download this data structure, we precomputed this transitive closure as an [adjacency list (click here if you don't know what that is)](networkx.readthedocs.io/en/latest/reference/readwrite.adjlist.html)  using the parent-child relationship, and stored it in a file that can be easily downloaded from the terminology server API. [This link points to the file](http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/transitive_closure/adjacency_list), and will automatically start a download if you click on it. 

In the two cells below, we've downloaded the adjacency list using `requests`, and saved it to a file 'children_subsumption.adjlist'. The file extension `.adjlist` is just plain text, so there's no extra processing required.

In [58]:
import requests
response=requests.get(
   "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/transitive_closure/adjacency_list"
)

In [59]:
with open('transitive_closure_adjacency_list.adjlist', 'w') as f:
   f.write(response.text)

## Start here if you don't want to download the adjacency list again.

In [1]:
import requests
import networkx as nx
from simplejson import loads, load,dumps,dump
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [60]:
G=nx.DiGraph()
DG = nx.read_adjlist('transitive_closure_adjacency_list.adjlist', create_using=G)

In [3]:
DG.number_of_nodes()

651034

In [4]:
DG.number_of_edges()

947772

### Child concepts of 'Actual medicinal product'. These are the Actual Medicinal Products in our drug hierarchy

In [5]:
amp_successors = DG.successors('10363901000001102')

In [6]:
len(amp_successors)

46793

### DM+D Hierarchy identifiers - SCTIDs of AMP,AMPP,VMP,VTM,VMPP

In [7]:
amp=10363901000001102

In [8]:
vmpp=8653601000001108

In [9]:
vmp=10363801000001108

In [10]:
vtm=10363701000001104

In [11]:
# 10364001000001104
ampp=10364001000001104

In [12]:
dmd_hierarchy_identifiers = ['10363901000001102', '8653601000001108', '10363801000001108', '10363701000001104', '10364001000001104']

### Hierarchy identifiers for dm+d used in the creation of Dimension 1 (amp,vmp,vtm)

In [13]:
dimension_1_dmd_hierarchy_identifiers = ['10363901000001102', '10363801000001108', '10363701000001104']

### Obtain the SCTIDs for the children of "9191801000001103 |Trade family (product)|", which aren't needed here

In [14]:
trade_family = DG.successors('9191801000001103')

In [ ]:
#### example using zinnat: 228011000001102
#### get the 3 direct parents of "228011000001102 |Zinnat 125mg tablets (GlaxoSmithKline) (product)|"

In [ ]:
zinnat_predecessors=DG.predecessors('228011000001102')

In [ ]:
non_trade_zinnat_predecessors = [x for x in zinnat_predecessors if x not in trade_family]

In [ ]:
non_dmd_predecessors_of_zinnat = [x for x in non_trade_zinnat_predecessors if x not in dimension_1_dmd_hierarchy_identifiers]

In [ ]:
# returns the only parent of zinnat in the drug hierarchy
non_dmd_predecessors_of_zinnat

#### The amp-vmp map is correct once we remove trade family hierarchy and dmd hierarchy identifiers from the predecessors list

In [15]:
amp_drugs_and_their_vmp_parents = []
for amp_drug in amp_successors:
    parents_of_amp = DG.predecessors(amp_drug)
    for parent_of_amp in parents_of_amp:
        if not  parent_of_amp in trade_family and not parent_of_amp in dmd_hierarchy_identifiers:
            amp_drugs_and_their_vmp_parents.append({'amp': amp_drug, 'corresponding_vmp': parent_of_amp}) 

In [16]:
print(len(amp_drugs_and_their_vmp_parents))
print(amp_drugs_and_their_vmp_parents[0])

47055
{'amp': '10701211000001107', 'corresponding_vmp': '329928008'}


In [ ]:
nx.ancestors(DG, '324506000')

In [17]:
pharma_sctid=373873005

In [18]:
# Get the children of "Virtual Medicinal Product"
vmp_successors = DG.successors(str(vmp))
vmp_successors[0]
# get the children of "Virtual Therapeutic Moiety"
vtm_successors = DG.successors(str(vtm))
# get the children of "Pharmaceutical/Biologic Product"
pharma_successors = DG.successors(str(pharma_sctid))

In [19]:
vtms = []
for amp_vmp_map in amp_drugs_and_their_vmp_parents:
    # parents_of_vmp = DG.predecessors(amp_vmp_map['corresponding_vmp'])
    ancestors_of_vmp = nx.ancestors(DG, amp_vmp_map['corresponding_vmp'])
    for ancestor in ancestors_of_vmp:
        if ancestor in vtm_successors and not ancestor in dmd_hierarchy_identifiers:
            vtms.append({'amp': amp_vmp_map['amp'], 'vmp': amp_vmp_map['corresponding_vmp'], 'vtm': ancestor})

In [20]:
print(len(vtms))

50646


In [21]:
full_dmd = []

for amp_vmp_vtm_map in vtms:
    ancestors_of_vtm = nx.ancestors(DG, amp_vmp_vtm_map['vtm'])
    for ancestor in ancestors_of_vtm:
        if ancestor in pharma_successors:
            full_dmd.append({'amp': amp_vmp_vtm_map['amp'], 
                             'vmp': amp_vmp_vtm_map['vmp'], 
                             'vtm': amp_vmp_vtm_map['vtm'], 
                             'drug_class': ancestor})

In [22]:
print(full_dmd[0])
print([key for key in full_dmd[0].keys()])
print([int(val) for val in full_dmd[0].values()])

{'vmp': '329928008', 'drug_class': '10363601000001109', 'amp': '10701211000001107', 'vtm': '125694008'}
['vmp', 'drug_class', 'amp', 'vtm']
[329928008, 10363601000001109, 10701211000001107, 125694008]


In [23]:
with open('dmd.json') as dmd_file:
    dmd_name_data = load(dmd_file)

In [49]:
concept_by_id_list_api = "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/concept_list_by_id/"
headers = {'Expect': '100-Continue'}
def get_concept_term_by_id_list(id_list):
    ids = ','.join(id_list)
    try:
        response = requests.post(concept_by_id_list_api,data={'sctid_list': ids}).text
    except Exception as e:
        print("Request failed at id {}. Retrying...".format(id_list))
        return get_concept_term_by_id_list(id_list)
    try:
        data = loads(response)
    except Exception as e:
        print(response)
    return data

get_concept_term_by_id_list([val for val in full_dmd[0].values()])

{'10363601000001109': 'UK product (product)',
 '10701211000001107': 'Meloxicam 15mg tablets (Somex Pharma) (product)',
 '125694008': 'Meloxicam (substance)',
 '329928008': 'Meloxicam 15mg tablet'}

In [40]:
# extract all of the ids from the dict, into a list, 
# then get their preferred terms from the SNOMED CT Termserver API
dmd_values = [list(obj.values()) for obj in full_dmd]
dmd_id_set=set(list(chain(chain(*dmd_values))))
print("number of sctids in this hierarchy: {}".format(len(dmd_id_set)))
dmd_name_data=get_concept_term_by_id_list(list(dmd_id_set))

{'54765002': 'Phenylephrine (substance)', '85417000': 'Autonomic drug', '30785611000001106': 'Nurofen Sinus Pain Relief 200mg/5mg tablets (Reckitt Benckiser Healthcare (UK) Ltd) (product)', '16036311000001108': 'Ibuprofen 200mg / Phenylephrine 5mg tablets', '10363601000001109': 'UK product (product)'}
58695


In [42]:
dmd_data_flattened = []
for dmd_datum in full_dmd:
    dmd_data_flattened.append(
    {'drug_class': dmd_name_data[dmd_datum['drug_class']] + ' | {}'.format(dmd_datum['drug_class']),
     'amp' : dmd_name_data[dmd_datum['amp']]  + ' | {}'.format(dmd_datum['amp']),
     'vmp': dmd_name_data[dmd_datum['vmp']] + ' | {}'.format(dmd_datum['vmp']),
     'vtm': dmd_name_data[dmd_datum['vtm']] + ' | {}'.format(dmd_datum['vtm'])
     })

In [43]:
# check consistency of the flattened dm+d data, 
# by confirming that the sctid concatenated with the pipe is correct for the preferred term,
# for any given row
dmd_data_flattened[0]

{'amp': 'Nurofen Sinus Pain Relief 200mg/5mg tablets (Reckitt Benckiser Healthcare (UK) Ltd) (product) | 30785611000001106',
 'drug_class': 'Autonomic drug | 85417000',
 'vmp': 'Ibuprofen 200mg / Phenylephrine 5mg tablets | 16036311000001108',
 'vtm': 'Phenylephrine (substance) | 54765002'}

In [33]:
# i save the dmd data to a file in case anything goes wrong (i.e. i write a destructive function that modifies this data)
with open('dmd_data_flattened.json', 'w') as f:
    dump(dmd_data_flattened, f)
    

In [46]:
from itertools import chain

In [44]:
columns_map = map(lambda x: x.keys(), dmd_data_flattened)
columns_list = chain.from_iterable([list(list(column)) for column in columns_map])
columns=list(set(list(columns_list)))
columns = ['amp', 'vmp', 'vtm', 'drug_class']

In [66]:
# Confirm whether a given row has the correct structure. 
# If the results of line 2 match (by column) those of line 1, we're good
print(columns)
list(map(lambda x: dmd_data_flattened[0].get(x,""),columns))

['vmp', 'drug_class', 'amp', 'vtm']


['Insulin lispro biphasic 50/50 100units/ml suspension for injection 3ml pre-filled disposable devices (product)',
 'UK product (product)',
 'Humalog Mix50 Pen 100units/ml suspension for injection 3ml pre-filled pen (Eli Lilly and Company Ltd)',
 'Human insulin product (product)']

In [ ]:
# write data to csv file
import csv
with open('dmd_data_full.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(columns)
    for input_row in dmd_data_flattened:
        csv_writer.writerow(list(map(lambda x: input_row.get(x,""),columns)))

# Teaching Case: DM+D dimensional table that includes AMPP and VMPP
+ This will be tricker than the above case, due to dm+d's hierarchy. See the image:

![dm+d hierarchy](dmd_hierarchy_with_reltypes.png "DM+D Hierarchy With relationship type")

### The predecessors of AMPP (Actual Medicinal Product Pack)

In [23]:
# relationship type id for the "Has VMP" relationship type, whose destination id is always a vmp successor.
has_vmp_rel_type_id=10362601000001103

In [24]:
# relationship type id for the "Has AMP" relationship type, whose destination id is always an amp successor.
has_amp_rel_type_id=10362701000001108

In [25]:
destination_ids_api = "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/destination_by_type_id/{}"
def get_destination_id_by_reltype(type_id):
    print(type_id)
    try:
        response = requests.get(destination_ids_api.format(type_id)).text
    except Exception as e:
        print("Request failed. Retrying...".format(type_id))
        print(response)
    try:
        data = loads(response)
    except Exception as e:
        print(response)
    return data

In [26]:
# ampp successors are drugs under the 'AMPP (Actual medicinal product pack)' sub-hierarchy of DM+D
ampp_successors = DG.successors(str(ampp))

In [27]:
'1329211000001109' in ampp_successors

True

In [28]:
# This returns a large dict, with the format
# {'ampp_drug_code_1':'amp_drug_code_1','ampp_drug_code_2':'amp_drug_code_2',...}
# The ampp drug points to its amp parent.
ampp_amp_map = get_destination_id_by_reltype(has_amp_rel_type_id)

10362701000001108


In [29]:
non_dmd_predecessors_of_ampp_drugs = []
for ampp in ampp_successors:
    non_dmd_predecessors_of_ampp_drugs.append(
        {
            'ampp': ampp,
            'amp': ampp_amp_map[ampp]
        }
    )

In [30]:
non_dmd_predecessors_of_ampp_drugs[0]

{'amp': '17617411000001104', 'ampp': '17617711000001105'}

### The VMP predecessors of vmpp

In [31]:
# vmpp successors are drugs under the 'VMPP (Virtual medicinal product pack)' sub-hierarchy of DM+D
print(vmpp)
vmpp_successors = DG.successors(str(vmpp))

8653601000001108


In [32]:
# This returns a large dict, with the format
# {'vmpp_drug_code_1':'vmp_drug_code_1','vmpp_drug_code_2':'vmp_drug_code_2',...}
# The vmpp drug points to its vmp parent.
vmpp_vmp_map = get_destination_id_by_reltype(has_vmp_rel_type_id)

10362601000001103


In [33]:
vmpp_successors[0]

'17981411000001109'

In [34]:
len(vmpp_vmp_map.keys())

27890

In [35]:
list(vmpp_vmp_map.keys())[0] in vmpp_successors

True

In [36]:
vmpp_vmp_map['10235211000001109'] == '10246111000001108'

True

In [37]:
non_dmd_predecessors_of_vmpp = []
for vmpp in vmpp_successors:
    non_dmd_predecessors_of_vmpp.append(
    {
            'vmpp': vmpp,
            'vmp': vmpp_vmp_map[vmpp]
    }
    )

In [38]:
non_dmd_predecessors_of_vmpp[0]

{'vmp': '17986511000001104', 'vmpp': '17981411000001109'}

In [39]:
ampp_drugs_with_vtms = []
for ampp in non_dmd_predecessors_of_ampp_drugs:
    for vtm in vtms:
        if vtm['amp'] == ampp['amp']:
            ampp_drugs_with_vtms.append(
                {'amp': ampp['amp'], 
                 'ampp': ampp['ampp'], 
                 'vmp': vtm['vmp'], 
                 'vtm': vtm['vtm']
                }
            )

In [41]:
ampp_drugs_with_vtms[10]

{'amp': '31138811000001105',
 'ampp': '31138911000001100',
 'vmp': '31141511000001103',
 'vtm': '7561000'}

In [43]:
vmpp_drugs_with_ampps = []
for vmpp in non_dmd_predecessors_of_vmpp:
    for ampp in ampp_drugs_with_vtms:
        if ampp['vmp'] == vmpp['vmp']:
            vmpp_drugs_with_ampps.append({'amp': ampp['amp'], 'ampp': ampp['ampp'], 'vtm': ampp['vtm'], 'vmp': vmpp['vmp'], 'vmpp': vmpp['vmpp']})

In [44]:
vmpp_drugs_with_ampps[0]

{'amp': '30794511000001107',
 'ampp': '30794611000001106',
 'vmp': '30799011000001103',
 'vmpp': '30794411000001108',
 'vtm': '11145411000001101'}

In [51]:
full_vmpp_dmd_with_drug_class = []

for ampp_vmpp_amp_vmp_vtm_map in vmpp_drugs_with_ampps:
    ancestors_of_vtm = nx.ancestors(DG, ampp_vmpp_amp_vmp_vtm_map['vtm'])
    for ancestor in ancestors_of_vtm:
        if ancestor in pharma_successors:
            full_vmpp_dmd_with_drug_class.append(
                {
                    'amp': ampp_vmpp_amp_vmp_vtm_map['amp'],
                    'ampp': ampp_vmpp_amp_vmp_vtm_map['ampp'],
                    'vmp': ampp_vmpp_amp_vmp_vtm_map['vmp'],
                    'vmpp': ampp_vmpp_amp_vmp_vtm_map['vmpp'],
                    'vtm': ampp_vmpp_amp_vmp_vtm_map['vtm'], 
                    'drug_class': ancestor
                })

In [52]:
full_vmpp_dmd_with_drug_class[0]

{'amp': '30794511000001107',
 'ampp': '30794611000001106',
 'drug_class': '312416005',
 'vmp': '30799011000001103',
 'vmpp': '30794411000001108',
 'vtm': '11145411000001101'}

In [53]:
# extract all of the ids from the dict, into a list, 
# then get their preferred terms from the SNOMED CT Termserver API
vmpp_dmd_values = [list(obj.values()) for obj in full_vmpp_dmd_with_drug_class]
vmpp_dmd_id_set=set(list(chain(chain(*vmpp_dmd_values))))
print("number of sctids in this hierarchy: {}".format(len(vmpp_dmd_id_set)))
vmpp_dmd_name_data=get_concept_term_by_id_list(list(vmpp_dmd_id_set))

number of sctids in this hierarchy: 129940


#### Regexp test cases. In order to extract company names, we'll need to parse these strings and get (only) the company name.

In [1]:
string_1="Ethanol 100% solution for injection 2ml ampoules (Alliance Healthcare (Distribution) Ltd)"
string_2="Manusept antibacterial hand rub (Medlock Medical) (product)"
string_3="Rapiscan 400micrograms/5ml solution for injection vials (Rapidscan Pharma Solutions EU Ltd) (product)"


In [55]:
vmpp_dmd_data_flattened = []
for dmd_datum in full_vmpp_dmd_with_drug_class:
    vmpp_dmd_data_flattened.append(
    {'drug_class': vmpp_dmd_name_data[dmd_datum['drug_class']] + ' | {}'.format(dmd_datum['drug_class']),
     'amp' : vmpp_dmd_name_data[dmd_datum['amp']]  + ' | {}'.format(dmd_datum['amp']),
     'vmp': vmpp_dmd_name_data[dmd_datum['vmp']] + ' | {}'.format(dmd_datum['vmp']),
     'ampp' : vmpp_dmd_name_data[dmd_datum['ampp']]  + ' | {}'.format(dmd_datum['ampp']),
     'vmpp': vmpp_dmd_name_data[dmd_datum['vmpp']] + ' | {}'.format(dmd_datum['vmpp']),
     'vtm': vmpp_dmd_name_data[dmd_datum['vtm']] + ' | {}'.format(dmd_datum['vtm'])
     })

In [56]:
vmpp_dmd_data_flattened[0]

{'amp': 'Lamberts Glucosamine Complete tablets (Lamberts Healthcare Ltd) | 30794511000001107',
 'ampp': 'Lamberts Glucosamine Complete tablets (Lamberts Healthcare Ltd) 120 tablet | 30794611000001106',
 'drug_class': 'Drug values (substance) | 312416005',
 'vmp': 'Generic Glucosamine Complete tablets (product) | 30799011000001103',
 'vmpp': 'Generic Glucosamine Complete tablets 120 tablet | 30794411000001108',
 'vtm': 'Chondroitin + Glucosamine sulphate (product) | 11145411000001101'}

In [57]:
columns_map = map(lambda x: x.keys(), vmpp_dmd_data_flattened)
columns_list = chain.from_iterable([list(list(column)) for column in columns_map])
columns=list(set(list(columns_list)))

# write data to csv file
import csv
with open('dmd_data_ampp_level.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(columns)
    for input_row in vmpp_dmd_data_flattened:
        csv_writer.writerow(list(map(lambda x: input_row.get(x,""),columns)))